## Type Stability in Struct Types

In [70]:
module Foo

struct Bar{TI <: Integer, TF <: AbstractFloat, F}
    N::TI
    W::Vector{TF}
    B::F
end

end #module



function (f::Foo.Bar)(x::T) where T <: AbstractFloat
    out = 0.0
    for i in 1:f.N
        out += f.W[i] * f.B[i](x)
    end
    return out
end

N = 1000
W = rand(N)
Bf = build_basis(N, fourier_sine)
Bp = build_basis(N, palais)


f = Foo.Bar(N, W, Bf)
g = Foo.Bar(N, W, Bp)

f(0.1)
g(0.1)
# @code_warntype f(0.1)
# @code_warntype g(0.1)


@time f(0.1)
@time g(0.1)


  0.000026 seconds (1 allocation: 16 bytes)
  0.000095 seconds (5.00 k allocations: 78.125 KiB)


-0.6222974665918023

# Unify Projection Methods and Interpolation

In [3]:
using ForwardDiff
using Plots
using LinearAlgebra

In [60]:
include("src/functionals.jl")
using SpecialFunctions: gamma

""" Function to compute the normalizing constant of the n-th 
Jacobi-based polynomial"""
function C(n)
    return (2^5 / (2n+5))  * (gamma(n+3) * gamma(n+3)) / (32 * gamma(n+5) * factorial(big(n)))
end


""" Function to compute the coefficients of the n-th 
Jacobi-based polynomial"""
function B(n)
    return [gamma(n+3) * binomial(n, m) * gamma(n+m+5) / (factorial(big(n)) * gamma(n+5) * gamma(m+3)) for m in 0:n]
end


"""Construct the n-th jacobi-based polynomial. Note that 
the degree of the polynomial is 2 larger than n, n<2.
"""
function jacobi_polynomial(n)
    # Compute Coefficients in the jacobi polynomial
    Cn = C(n)
    Bn = [Float64(bi / sqrt(Cn)) for bi in B(n)]
    
    # Create function
    function (x)
        out = 0.0
        for m in 0:n
            out += Bn[m+1] * (x-1)^m
        end
        return out * x * (1-x)
    end
end


function palais_odd(m::Integer)
    function f( x::Float64 )::Float64
        return (cos(2π*m*x) - 1) / (sqrt(2) * π * m)
    end
end

function palais_even(m::Integer)
    function f(x::Float64)::Float64
        return sin(2π*m*x) / (sqrt(2) * π * m)
    end
end


function palais(n)
    if n % 2 == 1
        return palais_odd(n ÷ 2 +1)
    else
        m = n ÷ 2
        return palais_even(n ÷ 2 + 1)
    end
end


function fourier_sine(n)
    function (x)
        return sqrt(2) * sin(π*n*x)
    end
end


function build_basis(N, basistype)
    if basistype === jacobi_polynomial
        return [basistype(n) for n in 0:N-1]
    end
    return [basistype(n) for n in 1:N]
end


""" Function Builder. Takes a vector of basis functions, and a set of weights,
and return a function taking the linear combinations of this at x."""
function build_function(weights, basis)
    @assert length(weights) == length(basis) "weights and basis needs to be same length"
    N = length(weights)
    function (x)
        out = 0.0
        for i in 1:N
            out += weights[i] * basis[i](x)
        end
        return out
    end
end

build_function

In [5]:
function build_basis(N, basistype)
    if basistype === jacobi_polynomial
        return [basistype(n) for n in 0:N-1]
    end
    return [basistype(n) for n in 1:N]
end

build_basis (generic function with 1 method)

In [32]:
include("src/projection.jl")

basis_palais

In [33]:
tuple(basis)

(Any[var"#122#124"(Core.Box(1)), var"#123#125"(Core.Box(1)), var"#122#124"(Core.Box(2)), var"#123#125"(Core.Box(2)), var"#122#124"(Core.Box(3))],)

In [41]:
(tuple(basis...))

LoadError: syntax: "..." expression outside call around In[41]:1

In [35]:
N = 5
basis = build_basis(N, palais)
W = rand(N)

for bi in basis
    @code_warntype bi(0.1)
end



basis = basis_palais(N)
f = build_function(W, tuple(basis...))
@code_warntype f(0.1)

Variables
  #self#::var"#212#215"{Int64}
  x::Float64

Body::Float64
1 ─ %1 = (2 * Main.π)::Core.Compiler.Const(6.283185307179586, false)
│   %2 = Core.getfield(#self#, :m)::Int64
│   %3 = (%1 * %2 * x)::Float64
│   %4 = Main.cos(%3)::Float64
│   %5 = (%4 - 1)::Float64
│   %6 = Main.sqrt(2)::Float64
│   %7 = Core.getfield(#self#, :m)::Int64
│   %8 = (%6 * Main.π * %7)::Float64
│   %9 = (%5 / %8)::Float64
└──      return %9
Variables
  #self#::var"#213#217"{Int64}
  x::Float64

Body::Float64
1 ─ %1 = (2 * Main.π)::Core.Compiler.Const(6.283185307179586, false)
│   %2 = Core.getfield(#self#, :m)::Int64
│   %3 = (%1 * %2 * x)::Float64
│   %4 = Main.sin(%3)::Float64
│   %5 = Main.sqrt(2)::Float64
│   %6 = Core.getfield(#self#, :m)::Int64
│   %7 = (%5 * Main.π * %6)::Float64
│   %8 = (%4 / %7)::Float64
└──      return %8
Variables
  #self#::var"#212#215"{Int64}
  x::Float64

Body::Float64
1 ─ %1 = (2 * Main.π)::Core.Compiler.Const(6.283185307179586, false)
│   %2 = Core.getfield(#self#, :m):

In [ ]:
Palais()

In [68]:
function find_projection_weights(f, basis, inner_product)
    return [inner_product(f, bi) for bi in basis]
end

function project(f, N, basistype)
    basis = build_basis(N, basistype)
    
    if basistype === palais
        weights = find_projection_weights(f, basis, palais_inner_product)
    else
        weights = find_projection_weights(f, basis, l2_inner_product)
    end
    return build_function(weights, basis)
end
    

project (generic function with 2 methods)

In [11]:
vec = Vector{Function}(undef, 2)
vec[1] = x -> x
isassigned(vec, 1)

true

In [10]:
isassigned(vec, 1)

true

In [69]:
f(x) = x^2

f (generic function with 1 method)

In [70]:
f_new = project(f, 10, palais)

#230 (generic function with 1 method)

In [72]:
@code_warntype project(f, 10, palais)

Variables
  #self#::Core.Compiler.Const(project, false)
  f::Core.Compiler.Const(f, false)
  N::Int64
  basistype::Core.Compiler.Const(palais, false)
  basis::Array
  weights::Any

Body::var"#230#231"{_A,_B,_C} where _C where _B where _A
1 ─      Core.NewvarNode(:(weights))
│        (basis = Main.build_basis(N, basistype))
│   %3 = (basistype === Main.palais)::Core.Compiler.Const(true, false)
│        %3
│        (weights = Main.find_projection_weights(f, basis, Main.palais_inner_product))
└──      goto #3
2 ─      Core.Compiler.Const(:(weights = Main.find_projection_weights(f, basis, Main.l2_inner_product)), false)
3 ┄ %8 = Main.build_function(weights, basis)::var"#230#231"{_A,_B,_C} where _C where _B where _A
└──      return %8


In [71]:
@code_warntype f_new(0.1)

Variables
  #self#::var"#230#231"{Array{Float64,1},Array{Function,1},Int64}
  x::Float64
  out::Any
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Any
1 ─       (out = 0.0)
│   %2  = Core.getfield(#self#, :N)::Int64
│   %3  = (1:%2)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = out::Any
│   %12 = Core.getfield(#self#, :weights)::Array{Float64,1}
│   %13 = Base.getindex(%12, i)::Float64
│   %14 = Core.getfield(#self#, :basis)::Array{Function,1}
│   %15 = Base.getindex(%14, i)::Function
│   %16 = (%15)(x)::Any
│   %17 = (%13 * %16)::Any
│         (out = %11 + %17)
│         (@_4 = Base.iterate(%3, %10))
│   %20 = (@_4 === nothing)::Bool
│   %21 = Base.not_int(%20)::Bo

In [63]:
projector = PalaisProjector(4)

Projector(4, Function[var"#222#225"{Int64}(1), var"#223#227"{Int64}(2), var"#222#225"{Int64}(2), var"#223#227"{Int64}(3)], palais_inner_product)

# Full Algorithm

In [ ]:
id(x) = x
ϕ = id
@printf "Iter %4d Error %12.10e\n" 0 l2_distance(q, r)

for i in 1:10
#     basis = basis_jacobi_polynomials(i)
    basis = basis_fourier_sine(N)

    ∇E = l2_gradient(q, r)
    dγ = project_palais(∇E, N)
#     dγ = project(∇E, basis)
#     dγ = interpolate(∇E, (i+2), rbf=gaussian)
    
    εmax = max_step_length(dγ)
    ε = backtracking(q, r, dγ, εmax, ρ=0.9, c=0.1, verbose=false)
#     println("Iter $i. L2 Inner Product", l2_inner_product(∇E, dγ))
#     ε = εmax / 2.
#     println("εmax: $εmax, ε: $ε, ||dγ|| = $(l2_norm(dγ))")
        
    γ(x) = x - ε * dγ(x)
    ϕ = ϕ ∘ γ
    r = Q_reparametrization(r, γ)

    @printf "Iter %4d Error %12.10e εmax: %12.10e ε: %12.10e ||dγ||: %12.10e\n" i l2_distance(q, r) εmax ε l2_norm(dγ)

    
    p1 = plot(∇E, Xfine, title="Iter $i: L2- and projected gradient", label="∇E")#, ylims=(-2., 2.))
    plot!(dγ, Xfine, label="dγ")
    
    p2 = plot(ϕ, Xfine, title="Iter $i: First Reparametrization", legend=false)
    plot!(γ, Xfine, linestyle=:dash, color="red", linewidth=0.5)
    plot!(ψ, Xfine, linestyle=:dash, color="black")
    
    
    p3 = plot_comparison(r, "r$i", X=0:0.02:1)#, xlims=(-0.1, 1.5))
    
#     display(plot_reparametrization(s1, s2, ϕ))
    
    p = plot(p1, p2, p3, layout=@layout[a b; c], size=(900, 600))
    push!(plots, p)
#     display(p)
end

In [7]:
include("src/reparametrization.jl")
include("src/functionals.jl")
include("src/transform.jl")

Q_reparametrization (generic function with 1 method)

In [8]:
function reparametrize(q, r; maxiter=20, tol=10)
    id(x) = x
    ψ = id
    
    for i in 1:10
        ∇E = l2_gradient(q, r)
        dγ = project(∇E, Projector)
        
        εmax = max_step_length(dγ)
        ε = backtracking(q, r, dγ, εmax, ρ=0.9, c=0.1, verbose=false) # Can probably be unified with ∇E
        
        γ(x) = x - ε * dγ(x)
        r = Q_reparametrization(r, γ)
        ψ = ψ ∘ γ
    end
    return ψ
end

reparametrize (generic function with 1 method)

In [9]:
ψ(t) = 0.5t^2 + 0.5t
s1(t) = [cos(2π*t), sin(2π*t)]
s2 = s1 ∘ ψ
q = Q_transform(s2)
r = Q_transform(s1)

#57 (generic function with 1 method)

In [10]:
reparametrize(q, r)

LoadError: UndefVarError: project not defined